<a href="https://colab.research.google.com/github/gbaghdadi/NeuromatchDeepLearningProject/blob/main/NMAproject2023_PytorchRegression_StringerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.model_selection import KFold


In [2]:
# @title NMA Figure settings
from matplotlib import rcParams

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
# @title NMA Data retrieval
import os, requests

fname = "stringer_spontaneous.npy"
url = "https://osf.io/dpqaj/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [6]:
# @title NMA Data loading

dat = np.load('stringer_spontaneous.npy', allow_pickle=True).item()
print(dat.keys())

dict_keys(['sresp', 'run', 'beh_svd_time', 'beh_svd_mask', 'stat', 'pupilArea', 'pupilCOM', 'xyz'])


In [7]:
# @title Shape of each data

print('neuralActivity dimension:   ', dat['sresp'].shape)
print('stat dimension:  ', dat['stat'].shape)
print('running speed dimension:  ', dat['run'].shape)
print('pupil area dimension:  ', dat['pupilArea'].shape)
print('pupil xy position dimension:  ', dat['pupilCOM'].shape)
print('faceSVD dimension:   ',dat['beh_svd_time'].shape)

neuralActivity dimension:    (11983, 7018)
stat dimension:   (11983, 1)
running speed dimension:   (7018, 1)
pupil area dimension:   (7018, 1)
pupil xy position dimension:   (7018, 2)
faceSVD dimension:    (7018, 1000)


In [8]:
# @title Input and Output of the Model

neural_activity = dat['sresp']    # Input
run_speed = dat['run']            # Output
pupil_area = dat['pupilArea']     # Output

print('Input dimension:  ', neural_activity.shape)
print('Output dimension:  ',run_speed.shape)

Input dimension:   (11983, 7018)
Output dimension:   (7018, 1)


In [9]:
# reshaping input and output
neural_activity = np.transpose(neural_activity)
runspeed = run_speed.squeeze()
pupilarea = pupil_area.squeeze()
print('Input dimension:  ', neural_activity.shape)
print('Output dimension:  ',runspeed.shape)

Input dimension:   (7018, 11983)
Output dimension:   (7018,)


In [10]:
# @title Dimension Reduction of the Input using PCA
pca_model = PCA(n_components=500)
reduced = pca_model.fit_transform(neural_activity)
component = pca_model.components_
neural_activity = reduced
print(neural_activity.shape)
# print(component.shape)

(7018, 500)


#Pytorch Regression Model: neural activity -> running speed